In [1]:
# RUN IT
# Display multiple interactive objects in one shell
# No Need for print function
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Chapter 5: Function

In [9]:
# one line recursive factorial()
def fac(n):
    """returns factorial of n"""
    return 1 if n < 2 else n * fac(n - 1)

fac(10)

# print out the doc
fac.__doc__
help(fac) # doc

3628800

'returns factorial of n'

Help on function fac in module __main__:

fac(n)
    returns factorial of n



In [1]:
# use reduce
from functools import reduce
from operator import mul
# mul == lambda x, y: x * y

def fac(n):
    if n == 0:
        return 1
    return reduce(mul, range(1, n + 1))
fac(10)

3628800

Higher-Order Functions

In [16]:
# take functions as arguments
sorted(['asdf','df','ddfd'], key = len)
# any one args function can be used as key

['df', 'asdf', 'ddfd']

3 common higher-order functions

In [21]:
# only calculate the factorial of odd numbers
list(map(fac, filter(lambda x: x % 2, range(11))))

[1, 6, 120, 5040, 362880]

In [22]:
# However, this is the better way to use listcomp
# Fuck!
[fac(n) for n in range(11) if n % 2]

[1, 6, 120, 5040, 362880]

In [23]:
a = [1, -1, 0]
all(a) # all elements in a must be True (none zero)
any(a) # any elements in a is True

False

True

Higher-order functions always combined with anonymous functions (lambda function)

In [25]:
# sort the list with reverse spelling order
stuff = ['apples', 'bread', 'kite', 'swarm', 'men']
sorted(stuff, key=lambda word: word[::-1])

['bread', 'kite', 'swarm', 'men', 'apples']

To write function-like objects, we need the `__call__()` method

In [46]:
import random

class Bag(object):
    
    def __init__(self, stuff):
        self._stuff = stuff
        random.shuffle(self._stuff)
    
    def pick(self):
        return random.choice(self._stuff)
    
    def __call__(self):
        return self.pick()

In [50]:
bb = Bag([1, 2, 3])
bb.pick()
bb()

1

3

Function Introspection 自我反省

In [60]:
# dir(bb)
', '.join(dir(bb))

'__call__, __class__, __delattr__, __dict__, __dir__, __doc__, __eq__, __format__, __ge__, __getattribute__, __gt__, __hash__, __init__, __le__, __lt__, __module__, __ne__, __new__, __reduce__, __reduce_ex__, __repr__, __setattr__, __sizeof__, __str__, __subclasshook__, __weakref__, _stuff, pick'

In [65]:
# list out all the function-only methods (object don't have)
class C(): pass
obj = C()
def f(): pass

set(dir(f)) - set(dir(obj))

{'__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__'}

In [3]:
# inspect.signature: retrieve information from parameters
from inspect import signature

def fn(a, *b, c=None, **d):
    pass

sig = signature(fn)
str(sig)

'(a, *b, c=None, **d)'

In [76]:
# freeze argument with functools.partial
# partial(fucntion, parameters`)
from operator import mul
from functools import partial
# mul == lambda x, y: x * y

double = partial(mul, 2)
list(map(double, range(11)))

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

### Chapter 6: Design of Functions

Case

### Classic Strategy

Design Diagram

<img src="files/img/classic.png">

In [12]:
from abc import ABC,abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity

# The Context
class Order:
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, "__total"):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}'
        return fmt.format(self.total(), self.due())

# The Strategy
# abstract base class
class Promotion(ABC):
    
    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""

# Sub-strategies
# subclass of The Strategy
class FidPromo(Promotion):
    
    def discount(self, order):
        return order.total() * 0.5 if order.customer.fidelity >= 1000 else 0

class BulkPromo(Promotion):
    
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

class LargeOrderPromo(Promotion): # third Concrete Strategy
    
    def discount(self, order): # kk
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

Passed

### Chapter 7: Function Decorators and Closures